# Web Programming in Java

## Spring MVC

* How to create a Spring MVC project:
    1. Create a new Web dynamic project named `classspring`.
    2. In the folder `WEB-INF/lib` add the jar files from Spring.
    3. In the folder `WEB-INF` add the files `spring-context.xml` and `web.xml`.
    4. Create a package `ca.edu.collegeX.classspring.controller` and, inside it, a class named `HelloWorldController` containing the following:

    ```java
    @Controller
    public class HelloWorldController {
        @RequestMapping("/helloWorldController")
        public String execute()	{
            System.out.println("Executing the logic with Spring MVC");
            return "ok";
        }
    }
    ```

    5. Don’t forget to add the imports (make this automatically using `Ctrl + 1`, for it’s a good opportunity to verify if the Spring libraries are already being “seen”):
    ```java
    import org.springframework.stereotype.Controller;
	import org.springframework.web.bind.annotation.RequestMapping;
    ```
    
    6. Create a folder inside `WEB-INF` named `views`.
    7. In the folder views, create a file `ok.jsp` with the following:
    
    ```html
    <html>
        <body>
			First class of Spring MVC!
		</body>
    </html>
    ```
    
    8. You can see the result in: 
       http://localhost:8080/classspring/helloWorldController.
       
       
## Creating a CRUD application for the Students System using Spring MVC

1. Modify the project named `classspring`, adding the packages:
    ```java
    ca.edu.collegeX.classspring.dao
    ca.edu.collegeX.classspring.jdbc
    ca.edu.collegeX.classspring.model
    ```
    
2. Add to each package the files `StudentDAO`, `Student` and `ConnectionFactory`. Download and also add the database `progweb` and the H2 jar file.
3. Let us start to better organize our view files. Inside the folder `views`, create a folder `students` to store the JSP files related to the model `Student`.
4. Create a JSP file named `form.jsp` with a form to insert students:

    ```jsp
     <form action="insertStudent" method="post">
      Name: <br /> <input type="text" name="name" /><br />
      Email: <br /> <input type="text" name="email" /><br />
      Address: <br /> <input type="text" name="address" /><br />
      <input type="submit" value="Insert">
    </form>
    ```
5. Create a class `StudentController` with an insertion method in the package `ca.edu.collegeX.classspring.controller`:

    ```java
    @Controller
	public class StudentController {
		@RequestMapping("insertStudent")
		public String insert(Student student) {
			  StudentDAO dao = new StudentDAO();
			  dao.insert(student);
			  return "student/added";
		}
	}
    ```

6. In `WEB-INF/views/student`, create a file `added.jsp` to confirm and insert a new record:
    ```html
    <html>
        <body>
            New student successfully added!
        </body>
	</html>
    ```
    
7. Using Spring, the direct access to the files in `WEB-INF` is not allowed and, consequently, we cannot access the form directly. Therefore, in the class `StudentController` we create a method to call the form:
    ```java
    @RequestMapping("newStudent")
	public String form() {
        return "student/form";
	}
    ```
    
8. Access the form in: `http://localhost:8080/classspring/newStudent`.

### Exercise
Implement a list of students that must be updated after insertions, updates, and removals. Each line of the table must have a link to `Change` and a link to `Remove`. This page must also have a link to the insertion page for new students that we already made.

* **Step 1**: to implement the list of students, you must modify the class `StudentController.java`, adding the following:
```java
@RequestMapping("listStudents")
public String list(Model model) {
    StudentDAO dao = new StudentDAO();
    model.addAttribute("students", dao.getList());
    return "student/list";
}
```

Now, create the file `list.jsp` with the following:

```jsp
<%@ taglib uri="http://java.sun.com/jsp/jstl/core" prefix="c"%>
<%@ taglib uri="http://java.sun.com/jsp/jstl/fmt" prefix="f"%>
<%@ page language="java" contentType="text/html; charset=UTF-8" pageEncoding="UTF-8"%>
<html>
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
        <title>List of Students</title>
    </head>
    <body>
        <table border=1>
            <tr>
                <td><b>Id</b></td>
                <td><b>Name</b></td>
                <td><b>E-mail</b></td>
                <td><b>Address</b></td>
            </tr>
            <c:forEach items="${students}" var="student">
                <tr>
                    <td>${student.id}</td>
                    <td>${student.name}</td>
                    <td>${student.email}</td>
                    <td>${student.address}</td>
                </tr>
            </c:forEach>
        </table>
    </body>
</html>
```

* **Step 2**: to redirect a request back to the list of students, you must modify the class `StudentController.java`, modifying the methods of each operation as in the following example:
          
```java
@RequestMapping("insert")
public String insert(Student student) {
    StudentDAO dao = new StudentDAO();
    dao.insert(student);
    return "redirect:listStudents";
}
```

* Step 3: to create `Remove` links in the list, besides of implementing the operation in the class `StudentController.java`, you need the following code:

```html
<td><a href="remove?id=${student.id}">Remove</a></td>
```

* **Step 4**: to create `Change` links in the list, besides implementing the operation in the class `StudentController.java`, you need the following code:

```html
<td><a href="showStudent?id=${student.id}">Change</a></td>
```

Also, create an action for the URL `showStudent` in the class `StudentController.java`. This action must get the student data according to the given id, and then put the data in fields in a JSP file so that the user may be able to modify it:

```java
@RequestMapping("showStudent")
public String show(Long id, Model model) {
    StudentDAO dao = new StudentDAO();
    model.addAttribute("student", dao.searchById(id));
    return "student/show";
}
```
The page `show.jsp` contains:

```jsp
<%@ page language="java" contentType="text/html; charset=UTF-8" pageEncoding="UTF-8"%>
<html>
	<head>
		<title> Change Student</title>
	</head>
	<body>
		<form action="changeStudent" method="POST">
            <input type="hidden" id="id" name="id" value="$ {student.id }"/><br />
            Name: <input type="text" id="name" name="name" value="${student.name }"/><br />	 		
            Email: <input type="text" id="email" name="email" value="${student.email }"/><br />
            Address: <input type="text" id="address" name="address" value="${student.address }" /><br />
            <input type="submit" value="Change"/>
		</form>
	</body>
</html>
```

## Validation

Change the file `form.jsp` as follows:

```jsp
<%@ taglib uri="http://java.sun.com/jsp/jstl/core" prefix="c" %>
<%@ taglib uri="http://www.springframework.org/tags/form" prefix="form" %>
<%@ page language="java" contentType="text/html; charset=UTF-8" pageEncoding="UTF-8"%>
<%@ page import="java.util.*, ca.edu.collegeX.classspring.dao.*, ca.edu.collegeX.classspring.model.*"%>
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
	<head>
		<title>Insertion of Students</title>
	</head>
	<body>
		<form name="insertStudent" action="insert" method="POST">
		Name: <input type="text" id="name" name="name" />
		<form:errors path="student .name" cssStyle="color:red" /> <br />		
		Email: <input type="text" id="email" name="email" />
		<form:errors path="student .email" cssStyle="color:red" /> <br />
		Address: <input type="text" id="address" name="address" />
		<form:errors path="student.address" cssStyle="color:red" /> <br />
		<input type="submit" value="Insert"/>
		</form>
	</body>
</html>
```

Notice that using `<form: errors...>` in the same line of the input (before `<br />`) eases to know to which field the message is related to.

In the file `StudentController.java` modify the code as highlighted below:

```java
package ca.edu.collegeX.classspring.controller;

import javax.validation.Valid;
import org.springframework.stereotype.Controller;
import org.springframework.validation.BindingResult;
import org.springframework.web.bind.annotation.RequestMapping;
import ca.edu.collegeX.classspring.dao.StudentDAO;
import ca.edu.collegeX.classspring.model.Student;

@Controller
public class StudentController {
	@RequestMapping("newStudent")
	public String form() {
		return "form";
	}

	@RequestMapping("insert")
	public String insert(@Valid Student student, BindingResult result) {
		if (result.hasErrors()) {
			return "form";
		}
		StudentDAO dao = new StudentDAO();
		dao.insert(student);
		return "added";
	}
}
```

In `Student.java` modify the code as below:

```java
package ca.edu.collegeX.classspring.model;

import javax.validation.constraints.NotNull;
import javax.validation.constraints.Size;

public class Student {
	private Long id;

	@NotNull @Size(min=5)
	private String name;

	@NotNull @Size(min=5)
	private String email;

	@NotNull @Size(min=5)
	private String address;
	
	public Long getId() {
		return id;
	}

	public void setId(Long id) {
		this.id = id;
	}

	public String getName() {
		return name;
	}

	public void setName(String name) {
		this.name = name;
	}

	public String getAddress() {
		return address;
	}

	public void setAddress(String address) {
		this.address = address;
	}

	public String getEmail() {
		return email;
	}

	public void setEmail(String email) {
		this.email = email;
	}	
}
```

### Some Notes about the file `Student.java`

**VALIDATION OF THE FIELDS:**
If we want to use only the annotation `@NotNull`, we will have a problem because the empty input in HTML by default submits an empty string which then go through the validation. Therefore, to work around this problem, we use the annotation `@Size` with `@NotNull`. We can still use the annotation `@NotEmpty` which returns the answer "may not be empty" resulting in the following code:

```java
public class Student {
	private Long id;

	@NotEmpty 
	private String name;

	@NotEmpty
	private String email;

	@NotEmpty
	private String address;
	.
	.
	.
```

**VALIDATION OF EMAIL:**
If we want to validate the field `email` by verifying if it is a valid email, we can use an annotation that verifies a regular expression. See the example:

```java
public class Student {
	private Long id;

	@NotEmpty 
	private String name;

	@Pattern(regexp="^[_A-Za-z0-9-]+(\\.[_A-Za-z0-9-]+)*@[A-Za-z0-9]+(\\.[A-Za-z0-9]+)*(\\.[A-Za-z]{2,})$")
	private String email;

	@NotEmpty
	private String address;
	.
	.
	.
```

Regular expressions are studied in a Formal Languages and Automata Theory Course. Our validation verifies if:
1. it there exists a word (or two words with a dot between them) before the symbol @;
2. it there exists the symbol @; 
3. it there exists one word (or still two or three words separated by dots) after the symbol @. See the detailed explanation:

| Expression            | Explanation      |
| :-:                   | :--- |
| <center>`^`</center>                   | <center>beginning of line</center>        |
| <center>`[_A-Za-z0-9-]+`</center>      | <center>this word must be formed by the characters between the brackets and must have one or more characters (indicated by the symbol `+`)</center>     |
| <center>`(\\.[_A-Za-z0-9-]+)*`</center>| <center>this word is optional, which is indicated by the parentheses and by the symbol `*`. This word must start with a dot followed by one or more of the characters between the brackets</center>  |
| <center>`@`</center>                   | <center>it must have a symbol `@`</center>         |
| <center>`[A-Za-z0-9]+`</center>        | <center>this word must be formed by the characters between the brackets and must have one or more characters (indicated by the symbol `+`)</center>       |
| <center>`(\\.[A-Za-z0-9]+)*`</center>  | <center>this word is optional, which is indicated by the parentheses including everything and followed by the symbol `*`. This word must start with a dot `.` followed by one or more of the characters between the brackets</center>    |
| <center>`(\\.[A-Za-z]{2,})`</center>   | <center>this word must start with a dot and must be formed by the characters between the brackets, with minimum length of 2</center>    |
| <center>`$`</center>                   | <center>end of line</center>  |

## Internationalization

Modify the file `Spring-context.xml` as below:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans" 
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
xmlns:context="http://www.springframework.org/schema/context"
xmlns:mvc="http://www.springframework.org/schema/mvc"
xsi:schemaLocation="http://www.springframework.org/schema/mvc 
    http://www.springframework.org/schema/mvc/spring-mvc-3.0.xsd
    http://www.springframework.org/schema/beans 
    http://www.springframework.org/schema/beans/spring-beans-3.0.xsd
    http://www.springframework.org/schema/context 
    http://www.springframework.org/schema/context/spring-context-3.0.xsd">
  <context:component-scan base-package="ca.edu.collegeX.classspring" />
  <mvc:annotation-driven />
  <bean id="messageSource" 
  class="org.springframework.context.support.ReloadableResourceBundleMessageSource">
  	<property name="basename" value="/WEB-INF/messages" />  
  </bean>

  <bean class="org.springframework.web.servlet.view.InternalResourceViewResolver">
    <property name="prefix" value="/WEB-INF/views/"/>
    <property name="suffix" value=".jsp"/>
  </bean>
</beans>
```

`messages.properties`:

```properties
#override @NotEmpty default message
#override @Pattern default message
NotEmpty.student.name=Name must be filled!
NotEmpty.student.email=Email must be filled!
NotEmpty.student.address=Address must be filled!
Pattern.student.email=Email must have the format: word@word.word!
```

Modify the file `Student.java` as below:

```java
package ca.edu.collegeX.classspring.model;

import javax.validation.constraints.Pattern;
import org.hibernate.validator.constraints.NotEmpty;

public class Student {
	private Long id;
	@NotEmpty(message="{student.name}")
	private String name;
	@NotEmpty(message="{student.email}")
	@Pattern(regexp="^[_A-Za-z0-9-]+(\\.[_A-Za-z0-9-]+)*@[A-Za-z0-9]+(\\.[A-Za-z0-9]+)*(\\.[A-Za-z]{2,})$",message="{student.email}")
	private String email;
	@NotEmpty(message="{student.address}")
	private String address;
	
	public Long getId() {
		return id;
	}

	public void setId(Long id) {
		this.id = id;
	}

	public String getName() {
		return name;
	}

	public void setName(String name) {
		this.name = name;
	}

	public String getAddress() {
		return address;
	}

	public void setAddress(String address) {
		this.address = address;
	}

	public String getEmail() {
		return email;
	}

	public void setEmail(String email) {
		this.email = email;
	}	
}
```